In [1]:
from sqlalchemy import create_engine
import os
import pandas as pd
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

def create_connection():

    load_dotenv()
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')

    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    return conn

# установите соединение с базой
conn = create_connection()

In [6]:
# from sqlalchemy import inspect

# inspector = inspect(conn)
# if 'clean_flats_churn' in inspector.get_table_names():
#     # flats_table.drop(engine)
#     print(True)
# else:
#     print(False)
# # flats_table.create(engine)

False


In [2]:
# conn.execute("ALTER TABLE clean_flats_churn DROP CONSTRAINT IF EXISTS unique_flat_constraint")
        

In [3]:
sql = '''select f.* 
    from clean_flats_churn as f
    join buildings as b on f.building_id=b.id'''

test = pd.read_sql(sql, conn)
test

ProgrammingError: (psycopg2.errors.UndefinedColumn) column f.building_id does not exist
LINE 3:     join buildings as b on f.building_id=b.id
                                   ^

[SQL: select f.* 
    from clean_flats_churn as f
    join buildings as b on f.building_id=b.id]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [4]:
data = pd.read_sql(
    '''select f.id,f.floor,f.is_apartment,f.kitchen_area,f.living_area,f.rooms,
    f.studio,f.total_area,f.price,f.building_id,
    b.build_year,b.building_type_int,b.latitude,b.longitude,
    b.ceiling_height,b.flats_count,b.floors_total,b.has_elevator 
    from flats as f
    join buildings as b on f.building_id=b.id''', conn)
data

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9,False,9.90,19.900000,1,False,35.099998,9500000,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,7,False,0.00,16.600000,1,False,43.000000,13500000,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,9,False,9.00,32.000000,2,False,56.000000,13500000,17821,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,1,False,10.10,43.099998,3,False,76.000000,20000000,18579,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,3,False,3.00,14.000000,1,False,24.000000,5200000,9293,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141357,141357,16,False,11.00,18.000000,1,False,42.000000,10500000,22455,2013,4,55.626579,37.313503,2.64,672,25,True
141358,141358,5,False,5.28,28.330000,2,False,41.110001,7400000,3162,1960,1,55.727470,37.768677,2.48,80,5,False
141359,141359,7,False,5.30,20.000000,1,False,31.500000,9700000,6513,1966,4,55.704315,37.506584,2.64,72,9,True
141360,141360,15,False,13.80,33.700001,2,False,65.300003,11750000,23952,2017,4,55.699863,37.939564,2.70,480,25,True


In [5]:
buff=data.copy()

In [6]:
buff.dtypes

id                     int64
floor                  int64
is_apartment            bool
kitchen_area         float64
living_area          float64
rooms                  int64
studio                  bool
total_area           float64
price                  int64
building_id            int64
build_year             int64
building_type_int      int64
latitude             float64
longitude            float64
ceiling_height       float64
flats_count            int64
floors_total           int64
has_elevator            bool
dtype: object

In [20]:
data=buff.copy()
data

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9,False,9.90,19.900000,1,False,35.099998,9500000,6220,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,7,False,0.00,16.600000,1,False,43.000000,13500000,18012,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,9,False,9.00,32.000000,2,False,56.000000,13500000,17821,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,1,False,10.10,43.099998,3,False,76.000000,20000000,18579,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,3,False,3.00,14.000000,1,False,24.000000,5200000,9293,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141357,141357,16,False,11.00,18.000000,1,False,42.000000,10500000,22455,2013,4,55.626579,37.313503,2.64,672,25,True
141358,141358,5,False,5.28,28.330000,2,False,41.110001,7400000,3162,1960,1,55.727470,37.768677,2.48,80,5,False
141359,141359,7,False,5.30,20.000000,1,False,31.500000,9700000,6513,1966,4,55.704315,37.506584,2.64,72,9,True
141360,141360,15,False,13.80,33.700001,2,False,65.300003,11750000,23952,2017,4,55.699863,37.939564,2.70,480,25,True


In [21]:
data = data.drop(columns=['id','studio','building_id'])
data

,floor,is_apartment,kitchen_area,living_area,rooms,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,9,False,9.90,19.900000,1,35.099998,9500000,1965,6,55.717113,37.781120,2.64,84,12,True
1,7,False,0.00,16.600000,1,43.000000,13500000,2001,2,55.794849,37.608013,3.00,97,10,True
2,9,False,9.00,32.000000,2,56.000000,13500000,2000,4,55.740040,37.761742,2.70,80,10,True
3,1,False,10.10,43.099998,3,76.000000,20000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,3,False,3.00,14.000000,1,24.000000,5200000,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141357,16,False,11.00,18.000000,1,42.000000,10500000,2013,4,55.626579,37.313503,2.64,672,25,True
141358,5,False,5.28,28.330000,2,41.110001,7400000,1960,1,55.727470,37.768677,2.48,80,5,False
141359,7,False,5.30,20.000000,1,31.500000,9700000,1966,4,55.704315,37.506584,2.64,72,9,True
141360,15,False,13.80,33.700001,2,65.300003,11750000,2017,4,55.699863,37.939564,2.70,480,25,True


In [22]:
def remove_duplicates(data: pd.DataFrame):
        feature_cols = data.columns.tolist()
        is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
        data = data[~is_duplicated_features].reset_index(drop=True)
        return data

data = remove_duplicates(data)
data

,floor,is_apartment,kitchen_area,living_area,rooms,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,9,False,9.90,19.900000,1,35.099998,9500000,1965,6,55.717113,37.781120,2.64,84,12,True
1,7,False,0.00,16.600000,1,43.000000,13500000,2001,2,55.794849,37.608013,3.00,97,10,True
2,9,False,9.00,32.000000,2,56.000000,13500000,2000,4,55.740040,37.761742,2.70,80,10,True
3,1,False,10.10,43.099998,3,76.000000,20000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,3,False,3.00,14.000000,1,24.000000,5200000,1971,1,55.808807,37.707306,2.60,208,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123932,16,False,11.00,18.000000,1,42.000000,10500000,2013,4,55.626579,37.313503,2.64,672,25,True
123933,5,False,5.28,28.330000,2,41.110001,7400000,1960,1,55.727470,37.768677,2.48,80,5,False
123934,7,False,5.30,20.000000,1,31.500000,9700000,1966,4,55.704315,37.506584,2.64,72,9,True
123935,15,False,13.80,33.700001,2,65.300003,11750000,2017,4,55.699863,37.939564,2.70,480,25,True


In [23]:
def remove_low_price_rows(data: pd.DataFrame):
    filtered_dataset = data[data['price'] > 800000]
    return filtered_dataset

def remove_high_price_rows(data: pd.DataFrame):
    filtered_dataset = data[data['price'] < 500000000]
    return filtered_dataset

def remove_outliers_iqr(data: pd.DataFrame):
    filtered_df = data.copy()
    num_cols = data.select_dtypes(['float','int']).columns
    threshold = 1.5
    for column in num_cols:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        filtered_df = filtered_df[
            (filtered_df[column] >= lower_bound)
            & (filtered_df[column] <= upper_bound)
        ]

    return filtered_df

# data = data.drop(columns="studio")
# data = remove_duplicates(data)
data = remove_low_price_rows(data)
data = remove_high_price_rows(data)
data = remove_outliers_iqr(data)

data

,floor,is_apartment,kitchen_area,living_area,rooms,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,9,False,9.90,19.900000,1,35.099998,9500000,1965,6,55.717113,37.781120,2.64,84,12,True
1,7,False,0.00,16.600000,1,43.000000,13500000,2001,2,55.794849,37.608013,3.00,97,10,True
2,9,False,9.00,32.000000,2,56.000000,13500000,2000,4,55.740040,37.761742,2.70,80,10,True
4,3,False,3.00,14.000000,1,24.000000,5200000,1971,1,55.808807,37.707306,2.60,208,9,True
5,9,False,0.00,0.000000,2,51.009998,8490104,2017,4,55.724728,37.743069,2.70,192,17,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123931,8,False,6.00,42.000000,3,64.000000,10800000,1971,4,55.740402,37.834579,2.64,428,9,True
123933,5,False,5.28,28.330000,2,41.110001,7400000,1960,1,55.727470,37.768677,2.48,80,5,False
123934,7,False,5.30,20.000000,1,31.500000,9700000,1966,4,55.704315,37.506584,2.64,72,9,True
123935,15,False,13.80,33.700001,2,65.300003,11750000,2017,4,55.699863,37.939564,2.70,480,25,True


In [24]:
display(data['floor'].describe())

count    97077.000000
mean         6.725043
std          4.502208
min          1.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         20.000000
Name: floor, dtype: float64

In [25]:
data.dtypes

floor                  int64
is_apartment            bool
kitchen_area         float64
living_area          float64
rooms                  int64
total_area           float64
price                  int64
build_year             int64
building_type_int      int64
latitude             float64
longitude            float64
ceiling_height       float64
flats_count            int64
floors_total           int64
has_elevator            bool
dtype: object

In [26]:
cat_features = data.select_dtypes(include=['bool','int'])
print(cat_features)
potential_binary_features = cat_features.nunique() == 2
print(potential_binary_features)
binary_cat_features = cat_features[
    potential_binary_features[potential_binary_features].index
]
print(binary_cat_features)
other_cat_features = cat_features[
    potential_binary_features[~potential_binary_features].index
]
print(other_cat_features)
num_features = data.select_dtypes(["float"])
print(num_features)

        floor  is_apartment  rooms     price  build_year  building_type_int  flats_count  floors_total  has_elevator
0           9         False      1   9500000        1965                  6           84            12          True
1           7         False      1  13500000        2001                  2           97            10          True
2           9         False      2  13500000        2000                  4           80            10          True
4           3         False      1   5200000        1971                  1          208             9          True
5           9         False      2   8490104        2017                  4          192            17          True
...       ...           ...    ...       ...         ...                ...          ...           ...           ...
123931      8         False      3  10800000        1971                  4          428             9          True
123933      5         False      2   7400000        1960        

In [27]:
from sklearn.preprocessing import OneHotEncoder

one_hot_drop = OneHotEncoder(drop='if_binary', sparse_output=False) 
drop_res = one_hot_drop.fit_transform(binary_cat_features)
drop_res = pd.DataFrame(drop_res, columns=one_hot_drop.get_feature_names_out())
print(drop_res.head())
print('shape: ', drop_res.shape) 

   is_apartment_True  has_elevator_True
0                0.0                1.0
1                0.0                1.0
2                0.0                1.0
3                0.0                1.0
4                0.0                1.0
shape:  (97077, 2)


In [30]:
print(other_cat_features)
from category_encoders import CatBoostEncoder

catboost_enc = CatBoostEncoder()

catboost_enc.fit_transform(other_cat_features, data['price']).head() 

        floor  rooms     price  build_year  building_type_int  flats_count  floors_total
0           9      1   9500000        1965                  6           84            12
1           7      1  13500000        2001                  2           97            10
2           9      2  13500000        2000                  4           80            10
4           3      1   5200000        1971                  1          208             9
5           9      2   8490104        2017                  4          192            17
...       ...    ...       ...         ...                ...          ...           ...
123931      8      3  10800000        1971                  4          428             9
123933      5      2   7400000        1960                  1           80             5
123934      7      1   9700000        1966                  4           72             9
123935     15      2  11750000        2017                  4          480            25
123936     16      1 

,floor,rooms,price,build_year,building_type_int,flats_count,floors_total
0,9,1,9500000,1965,6,84,12
1,7,1,13500000,2001,2,97,10
2,9,2,13500000,2000,4,80,10
4,3,1,5200000,1971,1,208,9
5,9,2,8490104,2017,4,192,17


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler_res = scaler.fit_transform(num_features)
print(scaler_res)
print(pd.DataFrame(scaler_res, columns=scaler.get_feature_names_out()))

[[ 0.60827543 -0.53330246 -1.01709429 -0.11055352  1.18846969 -0.35827117]
 [-2.70936057 -0.76367874 -0.52650543  0.64028708  0.02667521  2.40230742]
 [ 0.30667228  0.31141076  0.28079255  0.11088882  1.05841139  0.10182495]
 ...
 [-0.93325228 -0.52632134 -1.24065363 -0.23417067 -0.65405714 -0.35827117]
 [ 1.91522319  0.43008952  0.85832128 -0.27716954  2.25184997  0.10182495]
 [-0.16248842 -0.66594335 -0.83700465  1.29017061  0.57432822  0.4085557 ]]
       kitchen_area  living_area  total_area  latitude  longitude  ceiling_height
0          0.608275    -0.533302   -1.017094 -0.110554   1.188470       -0.358271
1         -2.709361    -0.763679   -0.526505  0.640287   0.026675        2.402307
2          0.306672     0.311411    0.280793  0.110889   1.058411        0.101825
3         -1.704016    -0.945187   -1.706402  0.775105   0.693070       -0.665004
4         -2.709361    -1.922541   -0.029086 -0.037010   0.933089        0.101825
...             ...          ...         ...       .